In [1]:
import pandas as pd

In [2]:
import nltk
import numpy as np
import re
import string

import seaborn as sns
import matplotlib.pyplot as plt
from nltk.chunk import ne_chunk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [3]:
from pandarallel import pandarallel
import spacy

In [4]:
from google.cloud import storage
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "nlp_finalproject"

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_finalproject connected.


In [5]:
df_final_NER = pd.read_parquet("gs://nlp_finalproject/NER_Clean.parquet", engine='pyarrow')
df_final_NER.head(5)

,date,title,text,year,cleaned,clean_title,ORG_Ent,Person_Ent
0,2020-02-26,LegalTech Artificial Intelligence Market 2019 ...,LegalTech Artificial Intelligence Market 2019 ...,2020,LegalTech Artificial Intelligence Market 2019 ...,LegalTech Artificial Intelligence Market 2019 ...,[LegalTech Artificial Intelligence Market Tec...,[]
4,2020-08-07,Two dead as AI Express flight skids off Kozhik...,Two dead as AI Express flight skids off Kozhik...,2020,Two dead as AI Express flight skids off Kozhik...,Two dead as AI Express flight skids off Kozhik...,"[AI Express, PostBeyond BygoneEpic PowerIn Ret...",[]
7,2021-02-25,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted ...,2021,MulticoreWare Inc. Becomes CEVA s Trusted Part...,MulticoreWare Inc. Becomes CEVA s Trusted Part...,"[MulticoreWare Inc., Trusted Partner for Imagi...",[]
8,2022-10-06,Healthcare Artificial Intelligence Market Anal...,\n\nHealthcare Artificial Intelligence Market ...,2022,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,[Healthcare Artificial Intelligence Market Ana...,[]
9,2020-04-29,Artificial Intelligence As A Service Market : ...,\n\nArtificial Intelligence As A Service Marke...,2020,Artificial Intelligence As A Service Market : ...,Artificial Intelligence As A Service Market : ...,"[Artificial Intelligence As A Service Market, ...",[]


In [6]:
pandarallel.initialize(progress_bar=True, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [9]:
df_final_NER['NER_title_clean'] = df_final_NER['clean_title'].apply(lambda x: re.sub(r'[^\w\s.,]', '', x))

In [10]:
df_final_NER['NER_title_clean'] = df_final_NER['NER_title_clean'].str.replace('\d+', '', regex=True)

In [11]:
df_final_NER['NER_title_clean'] = df_final_NER['NER_title_clean'].apply(lambda x: re.sub(r'{So}', '', x))

In [12]:
df_final_NER['NER_title_clean'] = df_final_NER['NER_title_clean'].str.strip()

In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.2 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### NER to Find ORG and Person in Titles

In [14]:
nlp = spacy.load("en_core_web_sm")

In [15]:
def extract_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    return entities

df_final_NER['title_ORG_Ent'] = df_final_NER['NER_title_clean'].parallel_apply(extract_entities)

In [ ]:
# %%time
# def extract_entities(text):
#     doc = nlp(text)
#     entities = [ent.text for ent in doc.ents if ent.label_ == "Person"]
#     return entities

# df_final_NER['Person_ORG_Ent'] = df_final_NER['NER_title_clean'].parallel_apply(extract_entities)

In [24]:
tmp = df_final_NER[['ORG_Ent','year']]
plot_tmp = tmp.explode('ORG_Ent')
ORG_counts = plot_tmp['ORG_Ent'].value_counts()

ORG_Ent
AI                  1115677
Microsoft            114279
Gray Media Group     102416
Google                98442
PRNewswire            50668
GPT                   38059
Amazon                32284
IBM                   28285
AP                    24415
ML                    23908
Name: count, dtype: int64

In [27]:
ORG_counts.head(25)

ORG_Ent
AI                         1115677
Microsoft                   114279
Gray Media Group            102416
Google                       98442
PRNewswire                   50668
GPT                          38059
Amazon                       32284
IBM                          28285
AP                           24415
ML                           23908
Apple                        23454
COVID                        22102
Android                      19819
Bard                         19304
EU                           17998
Altman                       16569
Intel                        16373
Artificial Intelligence      16098
Meta                         15386
Gray Media Group, Inc.       12308
The Associated Press         12216
Forecast                     11558
Nvidia                       11543
PredictionsHow               11363
Environ                      11355
Name: count, dtype: int64